# 拉普拉斯方法定轨道六根数

## 1. 已知三次观测的观测坐标，定出某一时刻的位置矢量和速度矢量
按照笔记的方法分别推出各个量

注意时间单位的选取!


In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import datetime
import time

import DDS_CMpack as DDS

%matplotlib inline

In [2]:
# astronomy constants
R_earth = 6371
au = 149597870
r_station_earth = 0.999102 #*R_earth
r_earth_sun = 1*au

time_unit = 806.81163 #806.8116 # SI 


In [3]:
# give 3 observed position
# position_fram = pd.DataFrame({'alpha':\
#                         np.array([142.935,157.274166667,171.817916667,183.134166667,\
#                                   208.640416667,219.102083333]),\
#                        'delta':\
#                        np.array([8.521111111,-2.395,-14.508333333,-23.56333333,-40.020277777,-44.998611111]),\
#                        'UT1_hours':np.array([21.575128333,21.60311055,21.631419722,\
#                                              21.654491388,21.714094166,21.742875555])})

position_fram = pd.read_csv('Echo1st.csv')

position_fram

,UT1_hours,alpha,delta,year,month,days
0,21.575128,142.935000,8.521111,1965,1,14
1,21.603111,157.274167,-2.395000,1965,1,14
2,21.631420,171.817917,-14.508333,1965,1,14
3,21.654491,183.134167,-23.563333,1965,1,14
4,21.714094,208.640417,-40.020278,1965,1,14
5,21.742876,219.102083,-44.998611,1965,1,14


In [4]:
# 计算观测数据的儒略日时间和恒星时
year = 1965; month = 1; days = 14;
UT1_hours = np.array([21.575128333,21.60311055,21.631419722,21.654491388,21.714094166,21.742875555])
jd_t = []
SG_t = []
for i in range(len(UT1_hours)):
    SG_s,jd_s = DDS.UTC2SG(year,month,days,UT1_hours[i])
    SG_t.append(SG_s); jd_t.append(jd_s);
    
SG_array = np.array(SG_t); 
SG_degree = SG_array%(3600*24)/(3600*24)*360

jd_array = np.array(jd_t)
delta_t = (UT1_hours - UT1_hours[0])*3600/time_unit

position_fram['deltaT'] = delta_t

delta_t

array([0.        , 0.12485688, 0.25117263, 0.35411859, 0.62006667,
       0.74848946])

In [5]:
# give the station's lam and phi
lam = 118.82091666
phi = 31.893611111


In [6]:
# 需要根据alpha和delta求出所需的测站到地心的RA,DEC
station_earth = []
for i in range(len(position_fram)):
    # give the station's RA and DEC then convert to station_earth
    RA_station = SG_degree[i] + lam
    DEC_station = phi
    station_earth.append(DDS.RADEC2xyz(r_station_earth,RA_station,DEC_station))

station_earth

[array([-0.81183633, -0.24592818,  0.52786921]),
 array([-0.81000791, -0.25188507,  0.52786921]),
 array([-0.80811365, -0.25789773,  0.52786921]),
 array([-0.80653683, -0.26278746,  0.52786921]),
 array([-0.80232652, -0.27537441,  0.52786921]),
 array([-0.80022302, -0.28142853,  0.52786921])]

In [7]:
position_fram

,UT1_hours,alpha,delta,year,month,days,deltaT
0,21.575128,142.935000,8.521111,1965,1,14,0.000000
1,21.603111,157.274167,-2.395000,1965,1,14,0.124857
2,21.631420,171.817917,-14.508333,1965,1,14,0.251173
3,21.654491,183.134167,-23.563333,1965,1,14,0.354119
4,21.714094,208.640417,-40.020278,1965,1,14,0.620067
5,21.742876,219.102083,-44.998611,1965,1,14,0.748489


In [8]:
# add Pj ,Qj to positon_fram_fil
Lt = []; Pt = []; Qt = []
Lambda = []; Miu = []; Niu = [];

#由 delta,alpha 算出 L = [Lambda,Miu,Niu]
for j in range(len(position_fram)):
    delta = position_fram['delta'].values[j]*2*np.pi/360 ; 
    alpha = position_fram['alpha'].values[j]*2*np.pi/360;

    L = [np.cos(delta)*np.cos(alpha),\
         np.cos(delta)*np.sin(alpha),\
         np.sin(delta)]
    
    Lambda.append(L[0]); 
    Miu.append(L[1]);
    Niu.append(L[2]);
    
#     Lambda.append(np.cos(delta)*np.cos(alpha))
#     Miu.append(np.cos(delta)*np.sin(alpha))
#     Niu.append(np.sin(delta))

# 由[Lambda,Miu,Niu] 和 测站-地心矢量归算出 P,Q
    Pj = L[2]*station_earth[j][0] - L[0]*station_earth[j][2]
    Qj = L[2]*station_earth[j][1] - L[1]*station_earth[j][2]
    
    Lt.append(L)
    Pt.append(Pj)
    Qt.append(Qj)
#     print(Lt,Pt,Qt)

position_fram_fil = pd.DataFrame({'alpha':\
                        np.array([142.935,157.274166667,171.817916667,183.134166667,\
                                  208.640416667,219.102083333]),\
                       'delta':\
                       np.array([8.521111111,-2.395,-14.508333333,-23.56333333,-40.020277777,-44.998611111]),\
                       'jd':jd_array,\
                        'SG_d': SG_degree,\
                        'deltaT':delta_t,\
                        'P':np.array(Pt),'Q':np.array(Qt),\
                        'Lambda':np.array(Lambda),\
                        'Miu':np.array(Miu),'Niu':np.array(Niu)})
    
Lt 

[[-0.7891439307993768, 0.5960674272555554, 0.14817381228628504],
 [-0.9215582970478929, 0.38598449379160416, -0.04178846365477428],
 [-0.9582566000732619, 0.13778102486286736, -0.25052081271577153],
 [-0.9152477093203553, -0.0501154444748697, -0.39976252051552885],
 [-0.6721154419353875, -0.36706452095992986, -0.6430586832976101],
 [-0.5487447629378402, -0.44598590319844417, -0.7070896402130082]]

In [9]:
position_fram_fil

,Lambda,Miu,Niu,P,Q,SG_d,alpha,delta,deltaT,jd
0,-0.789144,0.596067,0.148174,0.296272,-0.351086,78.032128,142.935000,8.521111,0.000000,-12769.600293
1,-0.921558,0.385984,-0.041788,0.520311,-0.193223,78.453011,157.274167,-2.395000,0.124857,-12769.599128
2,-0.958257,0.137781,-0.250521,0.708283,-0.008122,78.878811,171.817917,-14.508333,0.251173,-12769.597948
3,-0.915248,-0.050115,-0.399763,0.805554,0.131507,79.225833,183.134167,-23.563333,0.354119,-12769.596987
4,-0.672115,-0.367065,-0.643059,0.870732,0.370844,80.122323,208.640417,-40.020278,0.620067,-12769.594503
5,-0.548745,-0.445986,-0.707090,0.855495,0.434417,80.555226,219.102083,-44.998611,0.748489,-12769.593304


In [36]:
# 为循环重新format，取出需要的数据
Observe_time = len(position_fram)

obs = [0,1,5] # 取全部观测，做最小二乘法定初轨

# 取定t0 
index = 1
t0 = 0.15
# t0 = 0.5*(position_fram_fil['deltaT'].values[index] + position_fram_fil['deltaT'].values[index+2])
print(t0)
# set F0 and G0 to begin the loop

F0 = np.ones(Observe_time).tolist()
G0 = (position_fram_fil['deltaT'].values - t0).tolist()

FN0 = []; GN0 = []
P = []; Q = []
Lambda = []; Niu = []; Miu = []
tao = [];
for i in range(len(obs)): # 把要用的那几次观测数据提取出来：
    j = obs[i]
    FN0.append(F0[j])
    GN0.append(G0[j])
    P.append(position_fram_fil['P'].values[j])
    Q.append(position_fram_fil['Q'].values[j])
    Niu.append(position_fram_fil['Niu'].values[j])
    Miu.append(position_fram_fil['Miu'].values[j])
    Lambda.append(position_fram_fil['Lambda'].values[j])
    tao.append(position_fram_fil['deltaT'].values[j] - t0)
    

0.15


In [37]:
# 循环，解出F,G。

Nwind = 0
key = 0
F = [FN0]; G = [GN0]

while key == 0:
    
    # 在一次循环中:
    # 最终的目的是解(A,b)的增广矩阵 - 对应(x,y,z,vx,vy,vz)
    A = []
    b = []

    # we have : Lt, position_fram_fil
    ## 从F,G 解r0,v0
    for j in range(len(obs)): # 三次观测(现在的F,G已经只剩我们需要的那几次观测的值了)

        A.append([Niu[j]*F[Nwind][j],0,-1*Lambda[j]*F[Nwind][j],Niu[j]*G[Nwind][j],0,-1*Lambda[j]*G[Nwind][j]])
        A.append([0,Niu[j]*F[Nwind][j],-1*Miu[j]*F[Nwind][j],0,Niu[j]*G[Nwind][j],-1*Miu[j]*G[Nwind][j]])
        b.append(P[j])
        b.append(Q[j])

#################################################################        
    # 三次观测，解矩阵得到最终的结果
        
#     AA = np.array(A)
#     bb = np.array(b)
    
#     print(Nwind,b)
    
#     ans = np.linalg.solve(AA,bb) #ans 6 位对应(x,y,z,vx,vy,vz)

#     type(ans)
#     r0 = np.sqrt(ans[0]**2 + ans[1]**2 + ans[2]**2)
#     v0 = np.sqrt(ans[3]**2 + ans[4]**2 + ans[5]**2)
#     print(Nwind,':',r0,v0)
    
#################################################################
    # 多次测量、多资料问题的解法：
    
    AA = np.array(A)
    b = np.array(b)

#     print(np.dot(AA.T.copy(),AA))
#     print(np.dot(AA.T.copy(),b))

    ans = np.linalg.solve(np.dot(AA.T.copy(),AA),\
                             np.dot(AA.T.copy(),b))
    
    r0 = np.sqrt(ans[0]**2 + ans[1]**2 + ans[2]**2)
    v0 = np.sqrt(ans[3]**2 + ans[4]**2 + ans[5]**2)
    print(Nwind,':',r0,v0)
    
#################################################################
    # 算出新的F 和 G，为接下来的迭代做准备

    # r0,v0获得F,G
    FNw = []; GNw = [];
    for j in range(len(obs)):
#         print(j)
        FNw.append(DDS.F(r0,v0,tao[j]))
        GNw.append(DDS.G(r0,v0,tao[j]))
        
    F.append(FNw); G.append(GNw)

#################################################################
    # 判定deltaF or G的大小，并且决定最后是不是选取这一项。
    Nwind += 1
    delta_F = np.abs(np.array(F[Nwind]) - np.array(F[Nwind - 1]));
    delta_G = np.abs(np.array(G[Nwind]) - np.array(G[Nwind - 1]))
    big_delta = np.max([np.max(delta_F),np.max(delta_G)])
    
    print(delta_F,delta_G)
    print('--------------------')

    if big_delta < 1e-13:
        print('end')
        print(delta_F,delta_G)
        r0_a = np.array([ans[0],ans[1],ans[2]])
        v0_a = np.array([ans[3],ans[4],ans[5]])
        key = 1

# 地球半径是 6371 km

0 : 0.9983696628609504 0.04803021352691175
[0.01142998 0.00031806 0.18573778] [5.76544107e-04 2.66766193e-06 3.95764639e-02]
--------------------
1 : 1.5201560842824253 1.7805059864432353
[0.00756257 0.00022535 0.14457522] [3.64914331e-04 1.87899554e-06 3.06892758e-02]
--------------------
2 : 1.1038149946915161 0.4484819447165051
[0.00506175 0.00014476 0.07340725] [2.50448562e-04 1.21203693e-06 1.36467028e-02]
--------------------
3 : 1.3100379772862978 1.1466000317336082
[3.17382018e-03 9.37125713e-05 5.26656842e-02] [1.53854258e-04 7.82351912e-07 1.02672706e-02]
--------------------
4 : 1.1622403504518384 0.6577454972145051
[2.08751995e-03 6.04954116e-05 3.09587465e-02] [1.02389804e-04 5.05952786e-07 5.70526840e-03]
--------------------
5 : 1.2489461037927927 0.9502576873041725
[1.32289340e-03 3.88355265e-05 2.09160562e-02] [6.43532522e-05 3.24411581e-07 3.96532756e-03]
--------------------
6 : 1.190422411762199 0.7549008718137569
[8.58378298e-04 2.49998664e-05 1.29743909e-02] [4.19

--------------------
87 : 1.21199877308474 0.8278613102797459
[3.73034936e-14 1.11022302e-15 5.69655434e-13] [1.80411242e-15 1.04083409e-17 1.06692433e-13]
--------------------
88 : 1.2119987730833213 0.82786131027497
[2.13162821e-14 5.55111512e-16 3.25961480e-13] [1.02695630e-15 6.93889390e-18 6.09512441e-14]
--------------------
89 : 1.2119987730783341 0.8278613102584091
[7.47180096e-14 2.22044605e-15 1.14408483e-12] [3.63598041e-15 1.73472348e-17 2.13939977e-13]
--------------------
90 : 1.211998773079758 0.8278613102632492
[2.12052598e-14 6.66133815e-16 3.27626815e-13] [1.02695630e-15 6.93889390e-18 6.10622664e-14]
--------------------
91 : 1.2119987730735862 0.8278613102427039
[9.24815780e-14 2.77555756e-15 1.41642253e-12] [4.55191440e-15 2.42861287e-17 2.64677169e-13]
--------------------
92 : 1.211998773088484 0.8278613102923501
[2.23154828e-13 6.55031585e-15 3.41948692e-12] [1.09079412e-14 5.55111512e-17 6.39044373e-13]
--------------------
93 : 1.2119987730819293 0.82786131027

In [38]:
print(r0_a*R_earth,r0*R_earth,(r0-r_station_earth)*R_earth)

print(v0_a*R_earth/time_unit,v0*R_earth/time_unit)

print(t0*time_unit)

[-6960.62386303  -962.489749    3201.11196529] 7721.6441833233885 1356.3653413233883
[-1.0413131  -5.03433572 -4.03811399] 6.537219112463189
121.0217445


In [39]:
r0_a*R_earth

array([-6960.62386303,  -962.489749  ,  3201.11196529])

In [40]:
v0_a*R_earth/time_unit

array([-1.0413131 , -5.03433572, -4.03811399])

# 这一步就做完了，跳转到Laplace_Step2

# 以下是附录：

In [30]:
# 解矩阵 - 多资料
A = [[1,2,3],[4,5,6],[7,8,9],[10,20,30],[40,50,60],[70,80,90]]
AA = np.array(A)
b = np.array([2,3,4,5,6,7])

# AA = np.dot(AA.T,AA)
# b = np.dot(AA.T,b)
print(np.dot(AA.T.copy(),AA))
print(np.dot(AA.T.copy(),b))
# np.linalg.solve(AA,b)
answer = np.linalg.solve(np.dot(AA.T.copy(),AA),\
                         np.dot(AA.T.copy(),b))
answer
# np.linalg.lstsq(AA,b)

[[ 6666  7878  9090]
 [ 7878  9393 10908]
 [ 9090 10908 12726]]
[ 822 1011 1200]


array([ 0.01332092, -0.43258243,  0.45556514])

In [14]:
f = np.ones(4)
f

array([1., 1., 1., 1.])